In [1]:
%run ./sessions.ipynb

In [2]:
from pyspark.sql.functions import explode, col

In [3]:
def explode_data(df):
    df = df.withColumn("l", explode(df.l))
    return df.withColumn("vs", explode(df.l.vs))

In [4]:
def output_formatter(df):
    df = df.withColumnRenamed("hr", "hora") \
                  .withColumn("codigo_empresa", col("l.c")) \
                  .withColumn("codigo_linha", col("l.cl")) \
                  .withColumn("sentido_operacao", col("l.sl")) \
                  .withColumn("destino_linha", col("l.lt0")) \
                  .withColumn("origem_linha", col("l.lt1")) \
                  .withColumn("veiculos_circulacao", col("l.qv")) \
                  .withColumn("prefixo_veiculo", col("vs.p")) \
                  .withColumn("flag_acessivel", col("vs.a")) \
                  .withColumn("data_hora", col("vs.ta")) \
                  .withColumn("latitude", col("vs.py")) \
                  .withColumn("longitude", col("vs.px"))

    return df.drop("l","vs") 

In [5]:
def run_position_trusted_job():
    print("Obtendo sessão spark e lendo os dados da Raw.")
    spark = get_or_create_session_spark("space_shuttle")
    df = spark.read.format("delta").load("s3a://raw/position")
    print(f"Dados lidos! Tamanho do dataframe {df.count()}")
    
    if df != None or df.count() == 0:
        print("Iniciando a formatação dos dados.")
        df = explode_data(df)
        df = output_formatter(df)
        print("Dados formatados!")
        
        print("Realizando a escrita dos dados na camada Trusted.")
        df.write.format("delta").mode("append").save("s3a://trusted/position")
        print("Processamento raw to trusted finalizado com sucesso!")
    else:
        print("Dados não encontrados!")
    